In [1]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]


In [3]:
#Cleaning and formatting the data
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

#defining the tokenizer
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

#converting categorical variable into dummy/indicator variables
Y = pd.get_dummies(data['sentiment']).values

4472
16986


In [4]:
#spliting dataset to training and testing dataset
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [5]:
#defining the model
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
#training the model
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
 - 13s - loss: 0.4500 - acc: 0.8116
Epoch 2/10
 - 9s - loss: 0.3272 - acc: 0.8637
Epoch 3/10
 - 9s - loss: 0.2824 - acc: 0.8809
Epoch 4/10
 - 9s - loss: 0.2520 - acc: 0.9005
Epoch 5/10
 - 9s - loss: 0.2248 - acc: 0.9078
Epoch 6/10
 - 9s - loss: 0.2048 - acc: 0.9201
Epoch 7/10
 - 9s - loss: 0.1877 - acc: 0.9253
Epoch 8/10
 - 9s - loss: 0.1679 - acc: 0.9356
Epoch 9/10
 - 9s - loss: 0.1567 - acc: 0.9361
Epoch 10/10
 - 9s - loss: 0.1439 - acc: 0.9405


In [7]:
#validating the model
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))


score: 0.57
acc: 0.84


In [8]:
#Testing accuracy for positive and negetive sentiments
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 59.54692556634305 %
neg_acc 91.183879093199 %


In [9]:
#saving the mode
import pickle
model.save('sentiment_analyser_model.h5')
#saving the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
#testing the model with new sentence
new_input = ['so good.']
#vectorizing the input by the pre-fitted tokenizer instance
new_input = tokenizer.texts_to_sequences(new_input)
#padding the tweet to have exactly the same shape as `training` input
new_input = pad_sequences(new_input, maxlen=28, dtype='int32', value=0)
print(new_input)
sentiment = model.predict(new_input,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0  41 139]]
positive


In [12]:
import pickle
import os
#Saving the model
if not os.path.exists('models'):
    os.makedirs('models')
    
MODEL_PATH = "models/sentiment_model.sav"
pickle.dump(model, open(MODEL_PATH, 'wb'))